In [3]:
import datetime
import pandas as pd
from openbb_terminal.sdk import openbb

import quantstats as qs #pip install quantstats
import backtrader as bt #pip install backtrader

In [2]:
#pip install backtrader

     ------------------------------------ 410.1/410.1 kB 984.9 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
#pip install openbb

  Using cached openbb-2.2.0-py3-none-any.whl (7.4 MB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached detecta-0.0.5-py3-none-any.whl (12 kB)
  Using cached ccxt-2.7.8-py2.py3-none-any.whl (3.4 MB)
  Using cached Quandl-3.7.0-py2.py3-none-any.whl (26 kB)
  Using cached tradingview_ta-3.3.0-py3-none-any.whl (10 kB)
  Using cached alpha_vantage-2.3.1-py3-none-any.whl (31 kB)
  Using cached linearmodels-4.27-cp310-cp310-win_amd64.whl (1.6 MB)
  Using cached feedparser-6.0.10-py3-none-any.whl (81 kB)
  Using cached python_binance-1.0.16-py2.py3-none-any.whl (65 kB)
  Using cached fred-3.1-py3-none-any.whl
  Using cached pandas-1.5.3-cp310-cp310-win_amd64.whl (10.4 MB)
  Using cached dnspython-2.3.0-py3-none-any.whl (283 kB)
  Using cached streamlit-1.17.0-py2.py3-none-any.whl (9.3 MB)
  Using cached sentiment_investor-2.1.0-py3-none-any.whl (11 kB)
  Using cached ascii_magic-1.6-py3-none-any.whl (7.6 kB)
  Using cached finvizfinance-0.14.4-py3-none-any.whl (42 kB)
  Usi

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.1 which is incompatible.


In [1]:
#pip install quantstats

     ---------------------------------------- 41.3/41.3 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
#There’s an unsolved issue with Backtrader that prevents it from downloading data. That’s why you need the OpenBB SDK.

#Here’s a simple workaround.

#This function downloads the data from the OpenBB SDK, converts it to a CSV, and reads it in the Backtrader’s `YahooFinanceCSVData`.

def openbb_data_to_bt_data(symbol, start_date, end_date):
    
    df = openbb.stocks.load(symbol, start_date=start_date, end_date=end_date)

    fn=f'{symbol.lower()}.csv'
    df.to_csv(fn)

    return bt.feeds.YahooFinanceCSVData(
        dataname=fn,
        fromdate=datetime.datetime.strptime(start_date, '%Y-%m-%d'),
        todate=datetime.datetime.strptime(end_date, '%Y-%m-%d'),
    )

In [5]:
def last_day_of_the_month(any_day):
    # the last 28th of the month always exists. 4 days later, it is always the next month
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    # subtracking the current day from the next month brings up back one month
    return next_month - datetime.timedelta(days=next_month.day)

In [38]:
class MonthlyFlow(bt.Strategy):

    params = (
        ("end_of_month", 23),
        ("start_of_month", 7),
    )

    def __init__(self):
        self.order = None
        self.dataclose = self.datas[0].close

       # Indicators for the plotting show
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)
        
    def notify_order(self,order):
        # no more orders
        self.order = None

   
    def next(self):

        #get today's date, day of month, and last day of current month
        today = self.datas[0].datetime.date(0)
        day_of_month = today.day
        last_day_of_month = last_day_of_the_month(today)

        #if an order is pending, do nothing
        if self.order:
            return
        
        # check if we are in the market
        if not self.position:

            #if we are in the first week of the month, sell
            if day_of_month <= self.params.start_of_month:
                
                #Sell the whole portfolio
                
                self.order = self.order_target_percent(target=-1)       
                
                print(f'{today} - Created Sell {self.order.executed.size} shares of {self.datas[0]._name} at {self.dataclose[0]:.2f} on day {day_of_month}') 


            #if we are in the last week of the month, buy
            if day_of_month <= self.params.end_of_month:


                #Buy the whole portfolio
                self.order = self.order_target_percent(target=1)       

                print(f'{today} - Created Buy {self.order.executed.size} shares of {self.datas[0]._name} at {self.dataclose[0]:.2f} on day {day_of_month}')
        # if we are not in the market
       
        else:
            if self.position.size > 0:
                #if we are in not the last week of the month, close
                if  self.params.end_of_month <= day_of_month <= last_day_of_month.day:
                    
                    #Sell the whole portfolio
                    self.order = self.order_target_percent(target=0.0)       

                    print(f'{today} - Created close {self.order.executed.size} shares of {self.datas[0]._name} at {self.dataclose[0]:.2f} on day {day_of_month}')

            if self.position.size < 0:
                if not 1 <= day_of_month <= self.params.start_of_month:
                        
                    #Buy the whole portfolio
                    self.order = self.order_target_percent(target=0.0)
                    #if we are not in the first week of the month, close
                
                    print(f'{today} - Created close {self.order.executed.size} shares of {self.datas[0]._name} at {self.dataclose[0]:.2f} on day {day_of_month}')


In [22]:

data=openbb_data_to_bt_data('SPY', '2010-01-01', '2021-01-01')

Loading Daily data for SPY with starting period 2010-01-04.


In [1]:
#fetch data from OpenBB

data=openbb_data_to_bt_data('SPY', '2010-01-01', '2021-01-01')

#initialize backtrader
cerebro = bt.Cerebro(stdstats=False)

cerebro.adddata(data)
cerebro.broker.setcash(100000.0)

cerebro.addstrategy(MonthlyFlow)

cerebro.addobserver(bt.observers.Value)

cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='mydrawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='mytradeanalyzer')
cerebro.addanalyzer(bt.analyzers.Returns, _name='myreturns')
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='mytimereturn')

cerebro.broker.setcommission(commission=0.0)

backtest_results = cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Plot the result
cerebro.plot()

NameError: name 'openbb_data_to_bt_data' is not defined

In [5]:
pip install --upgrade setuptools

  Using cached setuptools-67.0.0-py3-none-any.whl (1.1 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.4.1
    Uninstalling setuptools-65.4.1:
      Successfully uninstalled setuptools-65.4.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
orange3 3.32.0 requires matplotlib>=2.2.5, which is not installed.
openbb 2.2.0 requires matplotlib<4.0.0,>=3.3.3, which is not installed.
tensorflow 2.8.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.1 which is incompatible.
openbb 2.2.0 requires setuptools<65.5.0, but you have setuptools 67.0.0 which is incompatible.


In [2]:

pip install matplotlib

     ---------------------------------------- 7.2/7.2 MB 2.0 MB/s eta 0:00:00
     -------------------------------------- 163.0/163.0 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openbb 2.2.0 requires setuptools<65.5.0, but you have setuptools 67.0.0 which is incompatible.
